In [13]:
from glob import glob
from tqdm import tqdm
import json
import re

In [26]:
rejected_words = [
    'kebutuhan',
    'berbeda',
    'bahwa',
    'Kode',
    'kode',
    'nomor',
    'RMXX,XXX',
    'kompleksitas',
    'listrik',
    'jawaban',
    'teknis',
    'berkualitas',
    'mencoba',
    'kampanye',
    'komunitas',
    'stabilitas',
    'Stabilitas',
    'metode',
    'pria',
    'butuh',
    'jadwal',
    'kasus',
    'otomatis',
    'populer',
    'bisnis',
    'probabilitas',
    'rusak',
    'kapasitas',
    'rutinitas'
]
rejected_words = set(rejected_words)

def check_indon(string):
    string = re.sub('[^A-Za-z ]+', ' ', string)
    return len(set(string.split()) & rejected_words) > 0

In [27]:
files = sorted(glob('data-oss_instruct-decontaminated/*.json'), key = lambda x: int(x.split('/')[-1].replace('.json', '')))
len(files)

70821

In [19]:
import tiktoken
enc = tiktoken.get_encoding("cl100k_base")

In [20]:
rejected_instructions = [
    'tidak dapat melakukan penulisan semula', 
    'maaf', 
    'not able to fulfill your request',
    'saya tidak dapat membantu',
    'dengan permintaan itu',
    'feel free to ask',
    'saya tidak bisa',
]

In [21]:
count, indon_count = 0, 0

indon_texts = []
for f in tqdm(files):
    try:
        with open(f) as fopen:
            data = json.load(fopen)
    except:
        continue

    if data['output'] is None:
        continue
        
    if any([r in data['instruction'].lower() for r in rejected_instructions]):
        continue

    indon_texts.append(data['instruction'])
    indon_texts.append(data['output'])

    count += 1

100%|██████████████████████████████████| 70821/70821 [00:01<00:00, 48727.09it/s]


In [22]:
len(indon_texts)

141148

In [23]:
print(indon_texts[1])

Berikut adalah implementasi fungsi convolution dalam bahasa C++:

```cpp
#include <iostream>
#include <vector>

std::vector<int> convolution(const std::vector<int>& a, const std::vector<int>& b) {
    int n = a.size();
    std::vector<int> c(2 * n - 1, 0);

    for (int i = 0; i < 2 * n - 1; i++) {
        for (int j = std::max(0, i - n + 1); j <= std::min(i, n - 1); j++) {
            c[i] += a[j] * b[i - j];
        }
    }

    return c;
}

int main() {
    std::vector<int> a = {1, 2, 3};
    std::vector<int> b = {4, 5, 6};

    std::vector<int> c = convolution(a, b);

    std::cout << "Hasil convolution: ";
    for (int i = 0; i < c.size(); i++) {
        std::cout << c[i] << " ";
    }
    std::cout << std::endl;

    return 0;
}
```

Output:
```
Hasil convolution: 4 13 28 27 18
```

Fungsi convolution menerima dua rangkaian a dan b sebagai parameter dan mengembalikan rangkaian hasil c setelah menjalankan operasi konvolusi. Implementasi ini menggunakan dua nested loop untuk melaku

In [24]:
with open('oss_instruct-decontaminated.texts', 'w') as fopen:
    for t in set(indon_texts):
        fopen.write(f'{json.dumps(t)}\n')

In [25]:
!cp oss_instruct-decontaminated.texts ~/ssd3/ctranslate2

In [28]:
mapping = {}
with open('/home/husein/ssd3/ctranslate2/oss_instruct-decontaminated.texts.requested') as fopen:
    for l in fopen:
        l = json.loads(l)
        mapping[l['src']] = l['r']
        
len(mapping)

140929

In [30]:
rejected = 0

with open('synthetic-oss_instruct-decontaminated.jsonl', 'w') as fopen_l:
    for f in tqdm(files):
        try:
            with open(f) as fopen:
                data = json.load(fopen)
        except:
            continue
            
        if data['output'] is None:
            continue
        
        output_ms = mapping.get(data['output'])
        instruction_ms = mapping.get(data['instruction'])
        
        indon_ins = False
        if check_indon(data['instruction']):
            indon_ins = True
        
        indon_output = False
        if check_indon(data['output']):
            indon_output = True
        
        rejected_ins = False
            
        if any([r in data['instruction'].lower() for r in rejected_instructions]):
            rejected_ins = True
        
        rejected_output = False
            
        if rejected_ins or rejected_output:
            rejected += 1
            
        data['indon_ins'] = indon_ins
        data['indon_output'] = indon_output
        data['instruction_ms'] = instruction_ms
        data['output_ms'] = output_ms
        data['rejected_ins'] = rejected_ins
        data['rejected_output'] = rejected_output
        
        fopen_l.write(f'{json.dumps(data)}\n')
        count += 1

100%|███████████████████████████████████| 70821/70821 [00:07<00:00, 9471.63it/s]
